In [1]:
from pysqlcipher3 import dbapi2 as sqlite
from os import environ
import pandas as pd
import numpy as np

In [2]:
keynow = environ['HOT_KEY']
conn = sqlite.connect('/opt/dssg-hot/data/hot-v3.db')
conn.execute('pragma key=\"x\''+keynow+'\'\"')

In [ ]:
print(conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())
dfttime = pd.read_sql_query("SELECT * FROM TTime", conn)
print(dfttime.head(2))

In [8]:
#print(dfttime.iloc[:,1:-1])
dfttime[dfttime.iloc[:,1:-1]<0] = np.nan
dfttime.fillna(method='ffill', inplace=True)
dfttime.set_index('aux1',inplace = True)
dfttime.drop('Time',inplace = True,axis = 1)
dictt = dfttime.to_dict('index')

In [21]:
dictt = dfttime.to_dict('index')


In [25]:
print(dictt['2018-01-010500']['HOV_12_12'])

27.0


In [32]:
dfrtime = pd.read_sql_query("SELECT * FROM reliability", conn)
print(dfrtime.columns)
dictr = dict(zip(dfrtime['auxind'],dfrtime['reliability']))
# print(dictr.keys())

Index(['tod', 'entr', 'exit', 'trip_id', 'reliability', 'auxind'], dtype='object')


In [9]:
dftot = pd.read_sql_query("SELECT trip_id,aux1,aux2,aux3,aux4,aux5,auxind FROM TOT", conn)

Index(['trip_id', 'aux1', 'aux2', 'aux3', 'aux4', 'aux5', 'auxind'], dtype='object')


In [29]:
dftot = dftot[dftot['aux3']!= 'GP_18_18' ]

                GP_12_12  HOV_12_12  GP_11_11  HOV_11_11  GP_11_12  HOV_11_12  \
aux1                                                                            
2018-01-010500      27.0       27.0      34.2       34.2      61.2       61.2   
2018-01-010505      27.0       27.0      34.2       34.2      61.2       61.2   

                GP_14_14  HOV_14_14  GP_14_15  HOV_14_15  ...  GP_16_23  \
aux1                                                      ...             
2018-01-010500      76.8       76.8     136.2      136.2  ...     503.4   
2018-01-010505      76.8       76.8     136.2      136.2  ...     503.4   

                HOV_16_23  GP_23_23  HOV_23_23  GP_21_23  HOV_21_23  GP_20_23  \
aux1                                                                            
2018-01-010500      503.4     102.6      102.6     132.0      132.0     286.8   
2018-01-010505      503.4     102.6      102.6     132.0      132.0     286.8   

                HOV_20_23  GP_19_23  HOV_19_23  


,trip_id,aux1,aux2,aux3,aux4,aux5,auxind
0,104021457,625,05_06,GP_05_06,HOV_05_06,2018-01-020625,62505_06
1,104021479,720,05_12,GP_05_12,HOV_05_12,2018-01-020720,72005_12


In [33]:
dftot['TT_GP'] = [dictt[x][y]/(60*60) for x,y in zip(dftot['aux5'],dftot['aux3'])]
dftot['TT_HOV'] = [dictt[x][y]/(60*60) for x,y in zip(dftot['aux5'],dftot['aux4'])]
dftot['TT_saving'] = dftot['TT_GP'] - dftot['TT_HOV'] 

In [34]:
dftot['reliability']=[dictr[x] for x in dftot['auxind']]

In [37]:
dftot.columns

Index(['trip_id', 'aux1', 'aux2', 'aux3', 'aux4', 'aux5', 'auxind', 'TT_GP',
       'TT_HOV', 'TT_saving', 'reliability'],
      dtype='object')

In [38]:
dffinal = dftot[['trip_id','TT_GP','TT_HOV','TT_saving','reliability']]
print(dffinal)

            trip_id     TT_GP    TT_HOV  TT_saving  reliability
0         104021457  0.103333  0.103333   0.000000     0.000000
1         104021479  0.222500  0.222500   0.000000     0.000000
2         104021501  0.162833  0.162833   0.000000     0.000000
3         104021513  0.137500  0.137167  -0.000333     0.000931
4         104021523  0.021000  0.021000   0.000000     0.000000
...             ...       ...       ...        ...          ...
16976129  151726679  0.016500  0.016500   0.000000     0.000000
16976130  151726680  0.044333  0.044333   0.000000     0.000000
16976131  151726681  0.044333  0.044333   0.000000     0.000000
16976132  151726682  0.038000  0.038000   0.000000     0.005400
16976133  151726683  0.015000  0.015000   0.000000     0.000917

[16954595 rows x 5 columns]


In [39]:
dffinal.to_sql('tt-table', con=conn, if_exists='fail')

In [40]:
print(conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall())



[('census',), ('hov',), ('trips',), ('bos',), ('trips_linked',), ('acct_stats',), ('acs',), ('frequency',), ('distance',), ('TTime',), ('reliability',), ('TOT',), ('tt-table',)]
